In [17]:
import pandas as pd
import os
DATA_PATH = "../calrecycle-data/"

## CalRecycle Waste Data

In [22]:
def filter_waste_origin_df(df, county):
    """Drop quarterly values in the csv and only keep yearly values
    Input:
        - calrecycle df
        - county (string)
    
    Ouput:
        - df with year, county, disposal ton
    """
    columns_to_keep = ["Report Year", "Diposal Ton", "County"]
    df["County"] = county
    missing_data = df["Report Year"].isnull()
    df = df[~missing_data]
    return df[columns_to_keep]

In [23]:
county_waste_production = []
for f in os.listdir(DATA_PATH):
    filename, ext = os.path.splitext(f)
    if ext == '.csv':
        county = filename.split("_waste_production")[0]
        county = county.replace("_", " ")
        print("[INFO] filtering county: ", os.path.join(DATA_PATH, f))
        try:
            df = pd.read_csv(os.path.join(DATA_PATH, f), skiprows=3, skipfooter=2)
            filtered_df = filter_waste_origin_df(df, county)
            county_waste_production.append(filtered_df)
        except Exception as e:
            print("[DEBUG] Skipping: ", county)

county_waste_production_df = pd.concat(county_waste_production)

[INFO] filtering county:  ../calrecycle-data/Butte_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Yolo_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/San_Benito_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/San_Diego_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Fresno_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Glenn_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Sutter_waste_production.csv
[DEBUG] Skipping:  Sutter
[INFO] filtering county:  ../calrecycle-data/Mendocino_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Santa_Cruz_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Los_Angeles_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Alpine_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Lassen_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/San_Luis_Obispo_waste_production.csv
[

/Users/mdong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


[INFO] filtering county:  ../calrecycle-data/Santa_Barbara_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/San_Joaquin_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Colusa_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Tulare_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Mono_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Sonoma_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Merced_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Kings_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Solano_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Sierra_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Yuba_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/Inyo_waste_production.csv
[INFO] filtering county:  ../calrecycle-data/San_Bernardino_waste_production.csv
[INFO] filtering county:  ../calrecycl

In [25]:
county_waste_production_df.head()

,Report Year,Diposal Ton,County
0,1995.0,200356.10,Butte
25,1996.0,197467.09,Butte
50,1997.0,178342.08,Butte
67,1998.0,200288.53,Butte
84,1999.0,211408.91,Butte


## Census Population Data